# Q# Interop with Qiskit

The modern QDK provides interoperability with Qiskit circuits built upon the core Q# compiler infrastructure.

This core enable integration and local resource estimation without relying on external tools. Users are able to estimate resources for their Qiskit circuits locally (see the [resource estimation with Qiskit sample notebook](../../estimation/estimation-qiskit.ipynb)), leveraging the Q# compiler's capabilities for analysis, transformation, code generation, and simulation. This also enables the generation of QIR from Qiskit circuits leveraging the [modern QDKs advanced code generation capabilities](https://devblogs.microsoft.com/qsharp/integrated-hybrid-support-in-the-azure-quantum-development-kit/).

This includes support for circuits with classical instructions available in Qiskit such as for loops, if statements, switch statements, while loops, and more.

Import the Q# module.

This enables the `%%qsharp` magic and initializes a Q# interpreter singleton.

In [ ]:
import qsharp

#### Qiskit circuits
We can create a simple circuit with a prepared state.

In [ ]:
from qiskit import QuantumCircuit
import numpy as np

circuit = QuantumCircuit(2, 2)
circuit.name = "state_prep"

# State vector to initialize: |ψ⟩ = (|0⟩ - |1⟩) / √2
circuit.initialize([1 / np.sqrt(2), -1 / np.sqrt(2)], 0)
circuit.h(0)
circuit.measure(0, 0)

circuit.prepare_state([1 / np.sqrt(2), -1 / np.sqrt(2)], 1)
circuit.h(1)
circuit.measure(1, 1)
circuit.draw(output="text")

We can run the circuit Q#'s backend

In [ ]:
from qsharp.interop.qiskit import QSharpSimulator

backend = QSharpSimulator()
job = backend.run(circuit)
counts = job.result().get_counts()
print(counts)

#### Parameterized Qiskit circuits
Some circuits require parameters as input. We can create a simple circuit that is parameterized.

In [ ]:
from typing import List

import numpy as np
from qiskit import QuantumCircuit
from qiskit.circuit import Parameter


def get_theta_range(samples: int) -> List[float]:
    return np.linspace(0, 2 * np.pi, samples)


def get_parameterized_circuit(n: int) -> QuantumCircuit:
    theta = Parameter("θ")
    n = 5
    qc = QuantumCircuit(n, 1)
    qc.h(0)
    for i in range(n - 1):
        qc.cx(i, i + 1)
    qc.barrier()
    qc.rz(theta, range(n))
    qc.barrier()

    for i in reversed(range(n - 1)):
        qc.cx(i, i + 1)
    qc.h(0)
    qc.measure(0, 0)
    return qc


def get_parameterized_circuits(n: int, theta_range: List[float]) -> List[QuantumCircuit]:
    qc = get_parameterized_circuit(n)
    qc.draw()
    theta = qc.parameters[0]
    circuits = [qc.assign_parameters({theta: theta_val}) for theta_val in theta_range]
    return circuits

Attempting to run without binding all input will generate an error in the job.

In [ ]:
from qsharp import QSharpError
from qsharp.interop.qiskit import QSharpSimulator
from qiskit.providers import JobStatus

circuit = get_parameterized_circuit(3)
backend = QSharpSimulator()
try:
    backend.qir(circuit)
except QSharpError as e:
    print(e)

Any parameters must be bound before we can run the circuit. This is done by setting the `params` argument to the `run` function.

In [ ]:
from qsharp.interop.qiskit import QSharpSimulator
from qiskit.providers import JobStatus

circuit = get_parameterized_circuit(3)
backend = QSharpSimulator()
# todo: should we support params={"θ": "0.5"}?
job = backend.run(circuit, params="0.5")
result = job.result()
assert job.status() == JobStatus.DONE
counts = result.get_counts()
print(counts)

#### Asychronous jobs

In [ ]:
from concurrent.futures import ThreadPoolExecutor

executor = ThreadPoolExecutor(max_workers=4)

Run the circuits

In [ ]:
num_samples = 256
theta_range = get_theta_range(num_samples)
circuits = get_parameterized_circuits(5, theta_range)

backend = QSharpSimulator()
job = backend.run(circuits, executor=executor)
job.wait_for_final_state(wait=0.5)
result = job.result()
print(job.status())
counts = job.result().get_counts()

Render the results using numpy

In [ ]:
import matplotlib.pyplot as plt

fig = plt.figure(figsize=(8, 6))
ax = fig.add_subplot(111)

theta_range = get_theta_range(num_samples)
ax.plot(theta_range, list(map(lambda c: c.get("0", 0), counts)), ".-", label="0")
ax.plot(theta_range, list(map(lambda c: c.get("1", 0), counts)), ".-", label="1")

ax.set_xticks([i * np.pi / 2 for i in range(5)])
ax.set_xticklabels(
    ["0", r"$\frac{\pi}{2}$", r"$\pi$", r"$\frac{3\pi}{2}$", r"$2\pi$"], fontsize=14
)
ax.set_xlabel("θ", fontsize=14)
ax.set_ylabel("Counts", fontsize=14)
ax.legend(fontsize=14)

## Classical instructions in circuits

### Run Qiskit with classical instructions
Qiskit has begun implementing some classical computation support as they expand their OpenQASM 3 support. These constructs, insofar as Qiskit can export them, can be consumed by Q#.

As an example, we can create a classical switch statement in Qiskit and look at the corresponding OpenQASM 3.

In [ ]:
from qiskit import ClassicalRegister, QuantumRegister
from qiskit.circuit import (
    Clbit,
    QuantumCircuit,
    Qubit,
)

from qsharp import QSharpError, TargetProfile

qreg = QuantumRegister(3, name="q")
creg = ClassicalRegister(3, name="c")
qc = QuantumCircuit(qreg, creg)
qc.h([0, 1, 2])
qc.measure_all(add_bits=False)

with qc.switch(creg) as case:
    with case(7):
        qc.x(0)
    with case(1, 2):
        qc.z(1)
    with case(case.DEFAULT):
        qc.cx(0, 1)

backend = QSharpSimulator()

print(backend.qasm3(qc))

Using that same circuit, we can generate QIR which is used to run on quantum hardware.

In [ ]:
backend = QSharpSimulator(target_profile=TargetProfile.Adaptive_RI)
print(backend.qir(qc))

In [ ]:
try:
    backend.qir(qc, target_profile=TargetProfile.Base)
except QSharpError as e:
    print(e)

### Run Qiskit circuit in new interpreter
The `run_qiskit` function can execute a qiskit circuit as a program using the output semantics of OpenQASM 3.0 (the circuit is converted to qasm3 before execution).

In [ ]:
from qiskit import QuantumCircuit

def ghz():
    circuit = QuantumCircuit(4, 3)
    circuit.name = "Qiskit Sample - 3-qubit GHZ circuit"
    circuit.h(0)
    circuit.cx(0, 1)
    circuit.cx(1, 2)
    circuit.measure([2, 1, 0], [1, 0, 2])
    return circuit


circuit = ghz()
print(circuit.draw(output="text"))
print(QSharpSimulator().run(circuit).result().get_counts())

## Errors
### Parse errors
Parse are exceptions. 

We can catch and handle parse errors prior to compilation.

In [ ]:
from qsharp.interop import QasmError, QiskitError

try:

except QiskitError as ex:
    print(ex)

### Runtime Errors
We can also catch and handle runtime errors, mapping the error back to the original code.

Here we'll run an example which tries to access the fifth element in an array with only two elements.

In [ ]:
from qsharp import QSharpError
from qsharp.interop import QasmError


try:
    circuit = QuantumCircuit(2)
    circuit.x(0)
    backend = QSharpSimulator()
    print(backend.run(circuit).result())
except QasmError as ex:
    print(ex)


The next example is for `QIR` code generation. When generating `QIR`, all output registers must be read into before generating QIR. Failure to do so results in a `QSharpError`

In [ ]:
circuit = QuantumCircuit(2, 2)
circuit.x(0)
circuit.measure(0, 1)
backend = QSharpSimulator()
try:
    # TODO: the error message is incorrect:
    # Unsupported target profile. Initialize Q# by running `qsharp.init(target_profile=qsharp.TargetProfile.Base)` before performing code generation.
    print(backend.qir(circuit))
except QSharpError as ex:
    print(ex)

print("Trying again with adaptive profile")
print(backend.qir(circuit, target_profile=TargetProfile.Adaptive_RI))

A runtime error that's not caught gets reported as a Python exception.

#### Unsupported language features, `QasmError`, and `QSharpError`
The modern QDK's interop with Qiskit is based on Qiskit's OpenQASM 3 support. Qiskit supports a subset of OpenQASM 3 features which may cause issues during conversion. If the Qiskit OpenQasm `Exporter` or OpenQasm parser don't support the feauture yet, a `QasmError` is raised prior to conversion. If there is a QASM parsing failure, this is likely an issue with the Qiskit libraries parsing or export functionality.

Any failure to transform the OpenQASM into Q#'s internal representation will throw a `QSharpError` to distinguish the scenarios.